In [ ]:
#Del b
for i, func in enumerate(func_list):

    #Splits 1/5 for testing and 4/5 for training
    X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=0.3)
    X_train_scaled, X_test_scaled, z_train_scaled, z_test_scaled = func(X_train, X_test, z_train, z_test)

    #finner beta
    beta = find_beta(X_train_scaled, z_train_scaled)
    #Her er z_ vår forutsigelse
    z_ = X_test_scaled @ beta

    #Find estimated of quality of fmodel fit 
    scores_OLS_scaled[i, 0] = MSE(z_test_scaled, z_)
    scores_OLS_scaled[i, 1] = R2(z_test_scaled, z_)

d1 = pd.DataFrame(scores_OLS_scaled, columns = ('MSE test', 'R2 test')) 
d = pd.concat([d1], axis =1)
names = ['Standard', 'Mean', 'MinMax', 'Robustscaler', 'None']
d['scaler'] = names
d[['scaler', 'MSE test','R2 test']].set_index('scaler')
display(d)

#del c
scores_OLS_boot = np.zeros((maxdegree, 2))
degrees = np.linspace(1, maxdegree, maxdegree, dtype=int)
metrics = {'degree': degrees,'error': [], 'bias': [], 'variance': []}

for degree in degrees:
    X = create_X(x, y, degree)
    X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=1/4)
    
    boot_n = 50

    bt_MSE, bt_R2, error, bias, var = bootstrap(X_train,z_train,X_test, z_test, iterations = boot_n) #bootstrapping the z-values to get a resampled set of the 'observed' data
    metrics['error'].append(error)
    metrics['bias'].append(bias)
    metrics['variance'].append(var)
    

    scores_OLS_boot[degree-1, 0] = bt_MSE
    scores_OLS_boot[degree-1, 1] = bt_R2
    #print(f"Errors for degree {degree}:")
    #print(f"MSE[Bootstrap] = {scores_OLS_boot[degree-1, 0]}")
    #print(f"MSE[w/0 resampling] = {scores_OLS_basic[degree-1, 0]}")
    #print(f"R2[Bootstrap] = {scores_OLS_boot[degree-1, 1]}")
    #print(f"R2[w/o resampling] = {scores_OLS_basic[degree-1, 1]}")
    #print(f"Improvement:")
    #print(f"RMSE: {abs(scores_OLS_boot[degree-1, 0] - scores_OLS_basic[degree-1, 0])}")
    #print(f"R2: {abs(scores_OLS_boot[degree-1, 1] - scores_OLS_basic[degree-1, 1])}\n")

    
estimates = pd.DataFrame(scores_OLS_boot, columns=['bt_MSE', 'bt_R2'])
bt_results = pd.concat([pd.DataFrame(metrics), estimates], axis = 1)
bt_results = bt_results.set_index('degree')
display(bt_results)
#error = scores_OLS_boot[maxdegree-1, 0]**2 #for each of the degrees, the MSE is used to get an error estimate
# finding the bias by subtractiong the mean of the bootstraped predictions from each test-data value,
# then calculating the mean of this - ideally it should be 0, indicating that the predictions are nomally distributed
# around the predictions

#print('Error:', error)
#print('Bias^2:', bias)
#print('Var:', variance)
degree7 = bt_results.loc[7]

plt.plot(degrees, scores_OLS_basic[:, 0], label="OLS w/o resampling")
plt.plot(degrees, scores_OLS_boot[:, 0], label="OLS w/ Bootstrap")
plt.xlabel("Degree")
plt.ylabel("MSE score")
plt.legend()
plt.title('Mean Squared Error as function of complexity')

plt.plot(degrees, bt_results['bias'], label="Boot bias")
plt.plot(degrees,  bt_results['error'], label="Boot error")
plt.plot(degrees,  bt_results['variance'], label="Boot variance")
plt.xlabel("Degree")
plt.ylabel("")
plt.legend()
plt.title('Mean Squared Error as function of complexity')

#del d
kfolds = 5
scores_OLS_CV = np.zeros((maxdegree, 1))
MSE_degrees = []
for degree in degrees:
    #denne gangen deler vi ikke inn i train og test
    #men vi kunne gjort det også, for enda et lag med vurdering
    #dette kan være nyttig for høye grader av maxdegree
    
    X = create_X(x, y, degree)
    #scores, z_, X_test, z_test = cross_validation(X, z, kfolds)
    scores,MSE_cv = cross_validation(X, z, kfolds)
    #print('scores: ', scores)
    scores_OLS_CV[degree-1, 0] = MSE_cv 
    #MSE_degrees.append(MSE_cv)
    #scores_OLS_CV[degree-1, 1] = R2(z_test, z_)
    '''print(f"Errors for degree {degree}:")
    print(f"RMSE[CV] = {scores_OLS_CV[degree-1, 0]}")
    print(f"RMSE[w/0 resampling] = {scores_OLS_basic[degree-1, 0]}")
    print(f"R2[CV] = {scores_OLS_CV[degree-1, 1]}")
    print(f"R2[w/o resampling] = {scores_OLS_basic[degree-1, 1]}")
    print(f"Improvement:")
    print(f"RMSE: {scores_OLS_basic[degree-1, 0] - scores_OLS_CV[degree-1, 0]}")
    print(f"R2: {scores_OLS_CV[degree-1, 1] - scores_OLS_basic[degree-1, 1]}\n")'''
 
datf = pd.DataFrame({'MSE_cv': scores_OLS_CV[:,0], 'degree': degrees}).set_index('degree')
display(pd.concat([datf, bt_results], axis=1))

plt.plot(degrees, scores_OLS_basic[:, 0], label="OLS w/o resampling")
plt.plot(degrees, scores_OLS_boot[:, 0], label="OLS w/ Bootstrap")
plt.plot(degrees, scores_OLS_CV[:, 0], label="OLS w/ CV")
plt.xlabel("Degree")
plt.ylabel("RMSE score")
plt.legend()

#del e
N = 5
#fra 10^-4 til 10^4, tar kun verdier 10^n, hvor n er et heltall
lmbda = np.logspace(-4, 4, N)

#scores_Ridge_boot = np.zeros((maxdegree, 2, N))
scores_Ridge_boot = np.zeros((maxdegree, 1)) #Bare droppa å lagre R2 ogsp for jeg vet ikke om det trengs
best_i = np.zeros(maxdegree, dtype=int)
for degree in degrees:
    X = create_X(x, y, degree)
    X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=0.2)
    
    #her utfører vi bootstrap
    B = len(z_train)
    z_train_boot = np.zeros(B)
    X_train_boot = np.zeros(X_train.shape)
    #for i in range(B):
     #   X_train_boot[i], z_train_boot[i] = bootstrap(X_train, z_train)
    mse_lambs, lmbd_outs = bootstrap(X_train, z_train, X_test, z_test, reg = 'ridge', lamb = True)    
    #itererer over alle lambda og bruker det feature matrise og data etter bootstrap
    '''for i, lmbda_ in enumerate(lmbda):
        beta = ridge_regression(X_train_boot, z_train_boot, lmbda_)
        z_ = X_test @ beta
        scores_Ridge_boot[degree-1, 0, i] = np.sqrt(MSE(z_test, z_))
        scores_Ridge_boot[degree-1, 1, i] = R2(z_test, z_)
        if i > 0:
            if scores_Ridge_boot[degree-1, 0, i] < scores_Ridge_boot[degree-1, 0, best_i[degree-1]]:
                best_i[degree-1] = i'''
        
    #her skal jeg legge inn slik at vi også gjør CV
    #lamb_df = pd.DataFrame({'mse_lamb': mse_lambs, 'lambdas': lmbd_outs})
    #display(lamb_df)
    scores_Ridge_boot[degree-1, 0] = min(mse_lambs)
    #print(f"Errors for degree {degree}:")
    #print(f"RMSE[Bootstrap] = {scores_Ridge_boot[degree-1, 0, best_i[degree-1]]}")
    #print(f"RMSE[w/0 resampling] = {scores_OLS_basic[degree-1, 0]}")
    #print(f"R2[Bootstrap] = {scores_Ridge_boot[degree-1, 1, best_i[degree-1]]}")
    #print(f"R2[w/o resampling] = {scores_OLS_basic[degree-1, 1]}")
    #print(f"Improvement:")
    #print(f"RMSE: {scores_OLS_basic[degree-1, 0] - scores_Ridge_boot[degree-1, 0, best_i[degree-1]]}")
    #print(f"R2: {scores_Ridge_boot[degree-1, 1, best_i[degree-1]] - scores_OLS_basic[degree-1, 1]}\n")

#print(scores_Ridge_boot)    
lamb_df = pd.DataFrame({'mse_lamb': scores_Ridge_boot[:, 0], 'degree': degrees})
display(lamb_df)

fig, ax = plt.subplots(int(N/2), int(N/2)+N%2, figsize= (12, 12))

for i in range(N):
    ax.flatten()[i].plot(degrees, scores_OLS_basic[:, 0], label="OLS w/o resampling")
    ax.flatten()[i].plot(degrees, scores_OLS_boot[:, 0], label="OLS w/ Bootstrap")
    ax.flatten()[i].plot(degrees, scores_OLS_CV[:, 0], label="OLS w/ CV")
    ax.flatten()[i].plot(degrees, scores_Ridge_boot[:, 0], label="Ridge w/ Bootstrap")
    ax.flatten()[i].set_xlabel("Degree")
    if i%(int(N/2)+1) == 0:
        ax.flatten()[i].set_ylabel("RMSE score")
    ax.flatten()[i].set_title(fr"$\lambda = ${lmbda[i]}")
    

handles, labels = ax.flatten()[i].get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')

N = 5
lmbda = np.logspace(-4, 4, N)

scores_Lasso_boot = np.zeros((maxdegree, 2, N))
best_i = np.zeros(maxdegree, dtype=int)
for degree in degrees:
    X = create_X(x, y, degree)
    X_train, X_test, z_train, z_test = train_test_split(X,z,test_size=0.2)
    #X_train, X_test = meanscaler(X_train), meanscaler(X_test)
    
    B = len(z_train)
    z_train_boot = np.zeros(B)
    X_train_boot = np.zeros(X_train.shape)
    
    mse_lambs, lmbd_outs = bootstrap(X_train, z_train, X_test, z_test, reg = 'lasso')
    scores_Lasso_boot[degree-1, 0] = min(mse_lambs)
    
lamb_df = pd.DataFrame({'mse_lamb': scores_Lasso_boot[:, 0], 'degree': degrees})
display(lamb_df)

fig, ax = plt.subplots(int(N/2), int(N/2)+N%2, figsize= (12, 12))

for i in range(N):
    ax.flatten()[i].plot(degrees, scores_OLS_basic[:, 0], label="OLS w/o resampling")
    ax.flatten()[i].plot(degrees, scores_OLS_boot[:, 0], label="OLS w/ Bootstrap")
    ax.flatten()[i].plot(degrees, scores_OLS_CV[:, 0], label="OLS w/ CV")
    ax.flatten()[i].plot(degrees, scores_Ridge_boot[:, 0], label="Ridge w/ Bootstrap")
    ax.flatten()[i].plot(degrees, scores_Lasso_boot[:, 0], label="Lasso w/ Bootstrap")
    ax.flatten()[i].set_xlabel("Degree")
    if i%(int(N/2)+1) == 0:
        ax.flatten()[i].set_ylabel("RMSE score")
    ax.flatten()[i].set_title(fr"$\lambda = ${lmbda[i]}")